#### Notebook setup

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import notebook_setup
import time
import warnings
import os, sys
from copy import deepcopy
from types import SimpleNamespace
from itertools import permutations

import matplotlib.pyplot as plt
import gym
import numpy as np
from numpy.linalg import norm
from tqdm.auto import tqdm, trange

from systems.trajplanning import TrajEnv as TrajEnvOG
from systems.plotting import plot_env_response
from rl import learn_rl, transform_rl_policy
from xform import (
    policy_transform,
    ab_xform_from_pseudo_matrix, pseudo_matrix_from_data,
    err_inv
)

### Parameters

In [ ]:
# Tarot T18 params
bp = BatteryParams(max_voltage=22.2)
mp = MotorParams(
    moment_of_inertia=5e-5,
    # resistance=0.27,
    resistance=0.081,
    k_emf=0.0265,
    # k_motor=0.0932,
    speed_voltage_scaling=0.0347,
    max_current=38.
)
pp = PropellerParams(
    moment_of_inertia=1.86e-6,
    use_thrust_constant=True,
    k_thrust=9.8419e-05, # 18-inch propeller
    # k_thrust=5.28847e-05, # 15 inch propeller
    k_drag=1.8503e-06, # 18-inch propeller
    # k_drag=1.34545e-06, # 15-inch propeller
    motor=mp
)
vp = VehicleParams(
    propellers=[pp] * 8,
    battery=bp,
    # angles in 45 deg increments, rotated to align with
    # model setup in gazebo sim (not part of this repo)
    angles=np.linspace(0, -2*np.pi, num=8, endpoint=False) + 0.375 * np.pi,
    distances=np.ones(8) * 0.635,
    clockwise=[-1,1,-1,1,-1,1,-1,1],
    mass=10.66,
    inertia_matrix=np.asarray([
        [0.2206, 0, 0],
        [0, 0.2206, 0.],
        [0, 0, 0.4238]
    ])
)
sp = SimulationParams(dt=0.001, g=9.81)

### Multirotor

#### Motor

In [ ]:
%matplotlib inline
# Plot motor speeds as a function of time and input voltage signal
plt.figure(figsize=(8,8))
motor = Motor(mp, sp)
for signal in [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]:
    speeds = []
    motor.reset()
    for i in range(200):
        speeds.append(motor.step(signal))
    plt.plot(speeds, label='%dV' % signal)
plt.legend(ncol=2)
plt.ylabel('Speed rad/s')
plt.xlabel('Time /ms')

In [ ]:
from multirotor.helpers import learn_speed_voltage_scaling

def make_motor_fn(params, sp):
    from copy import deepcopy
    params = deepcopy(params)
    params.speed_voltage_scaling = 1.
    def motor_step(signal):
        m = Motor(params, sp)
        for i in range(100):
            s = m.step(signal)
        return s
    return motor_step

print('Voltage = %.5f * speed' % (learn_speed_voltage_scaling(make_motor_fn(mp, sp))))

#### Propeller

In [ ]:
%matplotlib inline
# Plot propeller speed by numerically solving the thrust equation,
# *if* accurate propeller measurements are given in params
pp_ = deepcopy(pp)
pp_.use_thrust_constant = False # Set to true to just use k_thrust
prop = Propeller(pp_, sp)
plt.figure(figsize=(8,8))
speeds = np.linspace(0, 600, num=100)
for a in np.linspace(0, 10, 10, endpoint=False):
    thrusts = []
    for s in speeds:
        thrusts.append(prop.thrust(s, np.asarray([0, 0, a])))
    plt.plot(speeds, thrusts, label='%.1f m/s' % a)
plt.xlabel('Speed rad/s')
plt.ylabel('Thrust /N')
plt.title('Thrust with airspeed')
plt.legend(ncol=2)

#### Vehicle

In [ ]:
# Combine propeller/motor/vehicle to get vehicle.
# Take off simulation
m = Multirotor(vp, sp)
log = DataLog(vehicle=m) # convenient logging class
m.reset()
m.state *= 0 # set to zero, reset() sets random values
action = m.allocate_control(
    thrust=m.weight * 1.1,
    torques=np.asarray([0, 0, 0])
)
for i in range(500):
    m.step_speeds(action)
    log.log()
log.done_logging()
plt.plot(log.z)

### PID Controller

In [ ]:
# From PID parameters file
def get_controller(m: Multirotor):
    pos = PosController(
        1.0, 0., 0., 1., dt=m.simulation.dt, vehicle=m)
    vel = VelController(
        2.0, 1.0, 0.5, 1000., dt=m.simulation.dt, vehicle=m)
    att = AttController(
        [2.6875, 4.5, 4.5],
        0, 0.,
        1., dt=m.simulation.dt, vehicle=m)
    rat = RateController(
        [0.1655, 0.1655, 0.5],
        [0.135, 0.135, 0.018],
        [0.01234, 0.01234, 0.],
        [0.5,0.5,0.5], dt=m.simulation.dt, vehicle=m)
    alt = AltController(
        1, 0, 0,
        1, dt=m.simulation.dt, vehicle=m)
    alt_rate = AltRateController(
        5, 0, 0,
        1, dt=m.simulation.dt, vehicle=m)
    ctrl = Controller(
        pos, vel, att, rat, alt, alt_rate
    )
    return ctrl

In [ ]:
%matplotlib inline
m = Multirotor(vp, sp)
ctrl = get_controller(m)
log = DataLog(controller=ctrl)
for i in range(100):
    action = ctrl.step((1,1,1,0))
    log.log()
log.done_logging()

plt.plot(log.actions[:,0], ls=':', label='thrust')
lines = plt.gca().lines
plt.twinx()
for s, axis in zip(log.actions.T[1:], ('x','y','z')):
    plt.plot(s, label=axis + '-torque')
plt.legend(handles=plt.gca().lines + lines)

#### Attitude Angle Controller

In [ ]:
m = Multirotor(vp, sp)
fz = m.weight
att =  get_controller(m).ctrl_a
log = DataLog(vehicle=m, controller=att, other_vars=('err',))
for i in range(5000):
    ref = np.asarray([np.pi/18, 0, 0])
    # action is prescribed euler rate
    action = att.step(ref, m.orientation)
    # action = np.clip(action, a_min=-0.1, a_max=0.1)
    m.step_dynamics(np.asarray([0, 0, 0, *action]))
    log.log(err=att.err_p[0])
    log._actions[-1] = action
log.done_logging()

plt.plot(log.roll * 180 / np.pi)
plt.twinx()
plt.plot(log.actions[:,0], ls=':', label='Rate rad/s')

#### Attitude Rate Controller

In [ ]:
m = Multirotor(vp, sp)
fz = m.weight
ctrl = get_controller(m)
rat = ctrl.ctrl_r
att = ctrl.ctrl_a
log = DataLog(vehicle=m, controller=rat, other_vars=('err',))
for i in range(5000):
    ref = np.asarray([np.pi/18, 0, 0])
    rate = att.step(ref, m.orientation)
    action = rat.step(rate, m.euler_rate)
    action = np.clip(action, a_min=-0.1, a_max=0.1)
    m.step_dynamics(np.asarray([0, 0, 0, *action]))
    log.log(err=rat.err_p[0])
    log._actions[-1] = action
log.done_logging()

plt.plot(log.roll * 180 / np.pi)
plt.twinx()
plt.plot(log.actions[:,0], ls=':')

#### Altitude Controller

In [ ]:
m = Multirotor(vp, sp)
ctrl = get_controller(m)
alt = ctrl.ctrl_z
alt_rate = ctrl.ctrl_vz
log = DataLog(vehicle=m, controller=alt, other_vars=('thrust',))
for i in range(5000):
    ref = np.asarray([1.])
    rate = alt.step(ref, m.position[2:])
    action = alt_rate.step(rate, m.world_velocity[2:])
    action = np.clip(action, a_min=-2*m.weight, a_max=2*m.weight)
    m.step_dynamics(np.asarray([0, 0, action[0], 0,0,0]))
    log.log(thrust=action)
    #log._actions[-1] = action
log.done_logging()

plt.plot(log.actions.squeeze())
plt.twinx()
plt.plot(log.z, ls=':')

#### Position Controller

In [ ]:
m = Multirotor(vp, sp)
ctrl = get_controller(m)
pos = ctrl.ctrl_p
vel = ctrl.ctrl_v
rat = ctrl.ctrl_r
att = ctrl.ctrl_a
log = DataLog(vehicle=m, controller=pos, other_vars=('err', 'att_actions'))
for i in range(5000):
    ref = np.asarray([1.,0.])
    velocity = pos.step(ref, m.position[:2])
    angles = vel.step(velocity, m.velocity[:2])[::-1]
    rate = att.step(np.asarray([*angles, 0]), m.orientation)
    action = rat.step(rate, m.euler_rate)
    action = np.clip(action, a_min=-0.1, a_max=0.1)
    m.step_dynamics(np.asarray([0, 0, m.weight, *action]))
    log.log(err=pos.err_p[0], att_actions=action)
log.done_logging()

plt.plot(log.position[:,0])
plt.plot(log.err)
# plt.plot(log.position[:,1])
plt.twinx()
plt.plot(log.actions[:,0] * 180 / np.pi, ls=':')
plt.plot(log.pitch * 180 / np.pi, ls='-.')
# plt.plot(log.actions[:,0] * 180 / np.pi, ls=':')

## Trajectory Adaption

### Funcs

In [ ]:
def plot_disturbance(fn, ax, xlims, ylims, n=10, transpose=False):
    distx = np.zeros((n, n))
    disty = np.zeros_like(distx)
    xs = np.linspace(*xlims, num=n)
    ys = np.linspace(*ylims, num=n)
    for xi, x in enumerate(xs):
        for yi, y in enumerate(ys):
            vec = fn(0, (x,y))
            distx[yi,xi] = vec[0]
            if len(vec) > 1:
                disty[yi,xi] = vec[1]
    if transpose:
        distx, disty = disty, distx
    xx, yy = np.meshgrid(xs, ys)
    distmag = np.sqrt(distx**2 + disty**2)
    largest = np.max(distmag)
    distx = distx * distmag / (largest + 1e-6)
    disty = disty * distmag / (largest + 1e-6)
    if ax is None:
        plt.quiver(xx, yy, distx, disty, angles='xy')
    else:
        ax.quiver(xx, yy, distx, disty, angles='xy')

def evaluate_planning(env, agent, pos, plot=True, axes=None):
    poserr = -np.asarray(pos)
    x = env.reset([*poserr,*np.zeros_like(poserr)])
    ndim = env.ndim
    positions = []
    velocities = []
    rewards = []
    actions = []
    done = False
    while not done and env.t < 20:
        action = agent.predict(x, deterministic=True)[0]
        x, r, done, *_ = env.step(action)
        actions.append(action)
        positions.append(-x[:ndim])
        velocities.append(x[ndim:])
        rewards.append(r)
    actions = np.asarray(actions).T
    positions = np.asarray(positions).T
    velocities = np.asarray(velocities).T
    rewards = np.asarray(rewards)
    
    if plot:
        t = np.arange(velocities.shape[1]) * env.dt
        if axes is None:
            plt.figure(figsize=(8,4))
            _, (ax1, ax2) = plt.subplots(1,2)
        else:
            ax1, ax2 = axes
        plt.sca(ax1)
        if env.ndim==2:
            shortest_path =  np.asarray([[0,0], env._start_pos]).T
            plt.plot(positions[0], positions[1])
            plt.plot(shortest_path[0], shortest_path[1], ls='--', c='k')
            plot_disturbance(env.disturbance, None, (pos[0],0), (pos[1],0))
            plt.axis('equal')
            plt.title('Reward: %.2f' % sum(rewards))
            plt.xlabel('x /m',c='r')
            plt.ylabel('y /m',c='g')
        elif env.ndim==1:
            plt.plot(t, positions.squeeze())
            plot_disturbance(env.disturbance, None, (0,t[-1]), (pos[0],0), transpose=True)
            plt.xlabel('t /s')
            plt.ylabel('x /m')
        plt.sca(ax2)
        lines = []
        for name, v, c in zip('xyz', velocities, 'rgb'):
            l, = plt.plot(t, v, label='vel-'+name, c=c)
            lines.append(l)
        plt.ylabel('Velocity / m/s')
        plt.xlabel('Time /s')
        plt.twinx()
        for name, a, c in zip('xyz', actions, 'rgb'):
            plt.plot(t, a, label='$\Delta$'+name, ls=':', c=c)
        for name, p, c in zip('xyz', positions, 'rgb'):
            plt.plot(t, p, label=name, ls='-.', c=c)
        plt.ylabel('Waypoint / m')
        plt.legend(handles=lines +plt.gca().lines)
        plt.xlabel('Time / s')
        plt.tight_layout()
    return sum(rewards)


def adapt_traj(nominal, new, agent, steps=100, factor=50):
    P_s = pseudo_matrix_from_data(TrajEnv(nominal), steps * factor, agent, 'steps')
    P_t = pseudo_matrix_from_data(TrajEnv(new), steps, agent, 'steps')
    i = SimpleNamespace()
    i.A_s, i.B_s, i.A_t, i.B_t, i.F_A, i.F_B = \
        ab_xform_from_pseudo_matrix(P_s, P_t, TrajEnv().dt)
    i.state_xform, i.action_xform = \
        policy_transform((i.A_s, i.B_s), xformA=i.F_A, xformB=i.F_B)
    agent_new = transform_rl_policy(agent, i.state_xform, i.action_xform)
    return agent_new, i


def conditional_adapt(nominal, new, agent, factor=50, rtol=1.0):
    infos = []
    for steps in trange(100, 2000, 200, leave=False):
        nA_s, nB_s, nF_A, nF_B, nxxform, nuxform = 1,1,1,1,1,1
        dA_s, dB_s, dF_A, dF_B, dxxform, duxform = 1,1,1,1,1,1

        agent_new, i = adapt_traj(nominal, new, agent, steps, factor)
        infos.append(i)

        dA_s, nA_s = norm(i.A_s) / nA_s, norm(i.A_s)
        dB_s, nB_s = norm(i.B_s) / nB_s, norm(i.B_s)
        dF_A, nF_A = norm(i.F_A) / nF_A, norm(i.F_A)
        dF_B, nF_B = norm(i.F_B) / nF_B, norm(i.F_B)
        dxxform, nxxform = norm(i.state_xform) / nxxform, norm(i.state_xform)
        duxform, nuxform = norm(i.action_xform) / nuxform, norm(i.action_xform)

        if all([abs(1-d) <= rtol for d in (dA_s, dB_s, dF_A, dF_B, dxxform, duxform)]):
            break
    return agent_new, infos

In [ ]:
ndim = 2
class TrajEnv(TrajEnvOG):
    def __init__( self, disturbance=lambda t, x: 0., dt=1e-1, ndim=ndim, seed=0):
        super().__init__(disturbance=disturbance, dt=dt, ndim=ndim, seed=seed)

# Nominal case
class DummyAgent:
    
    def predict(self, u, *args, **kwargs):
        return np.zeros(ndim, np.float32), None

def make_disturbance(angle, mag=1):
    dx, dy = mag * np.cos(angle), mag * np.sin(angle)
    arr = np.asarray([dx, dy], np.float32)
    if ndim==1:
        arr = arr[:1]
    def dist(t, x):
        return arr
    return dist

### Experiments

In [ ]:
def nominal_disturbance(t, x):
    return np.asarray([0,0], np.float32)[:ndim]
angles = np.linspace(0, 2 * np.pi, num=8)
dists = (
    nominal_disturbance,
    *[make_disturbance(a) for a in angles],
)
pos = (5,5)[:ndim]

In [ ]:
# Learn nominal behavior
learn_kwargs = dict(
    steps=150_000, n_steps=1800, gamma=0.99, batch_size=300, learning_rate=5e-4
)
agents = []
for d in tqdm(dists, leave=False):
    agents.append(
        learn_rl(TrajEnv(d), **learn_kwargs, tensorboard_log=None)
    )
# agent_west = learn_rl(TrajEnv(west_disturbance), **learn_kwargs, tensorboard_log='TrajEnv/%dd/West' % ndim)
# agent_east = learn_rl(TrajEnv(east_disturbance), **learn_kwargs, tensorboard_log='TrajEnv/%dd/East' % ndim)
# agent_north = learn_rl(TrajEnv(north_disturbance), **learn_kwargs, tensorboard_log='TrajEnv/$dd/North' % ndim)
# agent_south = learn_rl(TrajEnv(south_disturbance), **learn_kwargs, tensorboard_log='TrajEnv/$dd/South' % ndim)

# agents = (
#     agent_nominal,
#     agent_west, agent_east,
#     # agent_north, agent_south
# )

#### Adaptation fidelity

In [ ]:
Ro, R, R_ = [], [], []
plt.figure(figsize=(8,16))
for j in trange(0, 8, 1, leave=False):
    agent = agents[j+1]
    env = TrajEnv(dists[j+1])
    angle = angles[j]
    rewards, ropt, rold, deltar, errs = [], [], [], [], []
    for i, ang in enumerate(tqdm(angles, leave=False)):
        dist = dists[i+1]
        agent_new, info = adapt_traj(env.disturbance, dist, agent, steps=100, factor=1)
        ro = evaluate_planning(TrajEnv(dist), agent, pos, plot=False)
        r_ = evaluate_planning(TrajEnv(dist), agents[i+1], pos, plot=False) 
        r = evaluate_planning(TrajEnv(dist), agent_new, pos, plot=False)
        rewards.append(r)
        ropt.append(r_)
        rold.append(ro)
        errs.append(e)
    Ro.append(np.roll(rold, -j))
    R.append(np.roll(rewards, -j))
    R_.append(np.roll(ropt, -j))
    plt.subplot(8,1,j+1)
    plt.title('Nominal angle: %.2f $^\circ$' % (angle * 180/np.pi))
    plt.xlabel('Disturbance angle /$^\circ$')
    plt.ylabel('Reward')
    plt.plot(angles * 180/np.pi, rewards, label='$r_{adapted}$', c='c')
    # plt.plot(angles * 180/np.pi, rold, label='$r_{applied}$', c='b', ls='-.')
    plt.plot(angles * 180/np.pi, ropt, label='$r_{opt}$', c='r', ls=':')
    plt.axvline(x=angle * 180/np.pi, c='k')
plt.legend(handles=plt.gca().lines)
plt.tight_layout()

In [ ]:
plt.figure(figsize=(8,3))
plt.xlabel('Relative disturbance angle $^\circ$')
plt.ylabel('Reward')
RRo5 = [np.roll(r, 0) for i,r in enumerate(Ro)]
RR5 = [np.roll(r, 0) for i,r in enumerate(R)]
RR_5 = [np.roll(r, 0) for i,r in enumerate(R_)]
plt.plot(angles * 180/np.pi, np.mean(RR5, axis=0), label='$r_{adapted}$', c='c')
# plt.plot(angles * 180/np.pi, np.mean(RRo, axis=0), label='$r_{applied}$', c='m', ls='-.')
plt.plot(angles * 180/np.pi, np.mean(RR_5, axis=0), label='$r_{opt}$', c='r', ls=':')
plt.legend(handles=plt.gca().lines)

In [ ]:
plt.figure(figsize=(8,3))
plt.xlabel('Relative disturbance angle $^\circ$')
plt.ylabel('Reward')
RRo1 = [np.roll(r, 0) for i,r in enumerate(Ro)]
RR1 = [np.roll(r, 0) for i,r in enumerate(R)]
RR_1 = [np.roll(r, 0) for i,r in enumerate(R_)]
plt.plot(angles * 180/np.pi, np.mean(RR1, axis=0), label='$r_{adapted}$', c='c')
# plt.plot(angles * 180/np.pi, np.mean(RRo, axis=0), label='$r_{applied}$', c='m', ls='-.')
plt.plot(angles * 180/np.pi, np.mean(RR_1, axis=0), label='$r_{opt}$', c='r', ls=':')
plt.legend(handles=plt.gca().lines)

In [ ]:
plt.figure(figsize=(8,3))
plt.xlabel('Relative disturbance angle $^\circ$')
plt.ylabel('Reward')
RRo2 = [np.roll(r, 0) for i,r in enumerate(Ro)]
RR2 = [np.roll(r, 0) for i,r in enumerate(R)]
RR_2 = [np.roll(r, 0) for i,r in enumerate(R_)]
plt.plot(angles * 180/np.pi, np.mean(RR2, axis=0), label='$r_{adapted}$', c='c')
# plt.plot(angles * 180/np.pi, np.mean(RRo, axis=0), label='$r_{applied}$', c='m', ls='-.')
plt.plot(angles * 180/np.pi, np.mean(RR_2, axis=0), label='$r_{opt}$', c='r', ls=':')
plt.legend(handles=plt.gca().lines)

In [ ]:
plt.figure(figsize=(8,3))
plt.xlabel('Relative disturbance angle $^\circ$')
plt.ylabel('Reward')
plt.plot(angles * 180/np.pi, np.mean(RR5, axis=0), label='$r_{adapted, 0.2}$', c='c')
plt.plot(angles * 180/np.pi, np.mean(RR1, axis=0), label='$r_{adapted,1.0}$', c='m')
plt.plot(angles * 180/np.pi, np.mean(RR2, axis=0), label='$r_{adapted,5.0}$', c='y')
# plt.plot(angles * 180/np.pi, np.mean(RRo, axis=0), label='$r_{applied}$', c='m', ls='-.')
plt.plot(angles * 180/np.pi, np.mean(RR_1, axis=0), label='$r_{opt}$', c='r', ls=':')
plt.legend(handles=plt.gca().lines)

In [ ]:
def new_disturbance(t, pos):
    return np.asarray([0.01, 0.7], np.float32)
new_dist = new_disturbance
P_s = pseudo_matrix_from_data(TrajEnv(east_disturbance), 5000, agent_east, 'steps')
P_t = pseudo_matrix_from_data(TrajEnv(new_dist), 100, agent_east, 'steps')
A_s, B_s, A_t, B_t, F_A, F_B = ab_xform_from_pseudo_matrix(P_s, P_t, TrajEnv().dt)
state_xform, action_xform = policy_transform((A_s, B_s), xformA=F_A, xformB=F_B)
agent_new = transform_rl_policy(agent_east, state_xform, action_xform)

In [ ]:
from systems.base import SystemEnv
import control
class DDTraj(SystemEnv):
    def __init__(self, sys):
        super().__init__(sys, dt=TrajEnv().dt)
        self.observation_space = TrajEnv().observation_space
        self.action_space = TrajEnv().action_space
        self.q = np.eye(ndim * 2)
        self.r = np.eye(ndim)
        self.period = TrajEnv().period
    def step(self, action):
        x,r,d,i = super().step(action)
        d = self.dt * self.n >= self.period
        return x,r,d,i
C_s = np.eye(len(A_s))
D_s = np.zeros_like(B_s)
sys = control.ss(A_s, B_s, C_s, D_s)
x0 = np.concatenate(([-5] * ndim, [0] * ndim)).astype(np.float32)
plot_env_response(DDTraj(sys),x0, agent_nominal, max_steps=100)

In [ ]:
# Env response plots (states/actions over time)
plt.figure(figsize=(12, 6))
for i, (ag, dist, name) in enumerate([
    (agent_nominal, nominal_disturbance, 'Nominal'),
    (agent_west, west_disturbance, 'West'),
    (agent_east, east_disturbance, 'East'),
    # (agent_north, north_disturbance, 'North'),
    # (agent_south, south_disturbance, 'South'),
]):
    x0 = np.concatenate(([-5] * ndim, [0] * ndim)).astype(np.float32)
    plt.subplot(3,2,i+1)
    plot_env_response(
        TrajEnv(dist),
        x0,
        ag, state_idx=(0,1)[:ndim], state_names='xy'[:ndim]
    )
    title = plt.gca().get_title()
    #plt.title(name + ' ' + title)
plt.tight_layout()

In [ ]:
# Plot nominal training
plt.figure(figsize=(6, 12))
for i, (ag, dist, name) in enumerate([
    (agent_nominal, nominal_disturbance, 'Nominal'),
    (agent_west, west_disturbance, 'West'),
    (agent_east, east_disturbance, 'East'),
    # (agent_north, north_disturbance, 'North'),
    # (agent_south, south_disturbance, 'South'),
]):
    ax1 = plt.subplot(5,2,2*i+1)
    ax2 = plt.subplot(5,2,2*i+2)
    evaluate_planning(TrajEnv(dist), ag, pos, axes=(ax1, ax2))
plt.tight_layout()

In [ ]:
# Plot nominal on disturbed cases
plt.figure(figsize=(12, 12))
ag_dist = list(zip(agents, dists))
for i, ((ag, old_dist), (ag_opt, new_dist)) in enumerate(permutations(ag_dist, 2)):
    ag_new, info = adapt_traj(old_dist, new_dist, ag)
    ax1, ax2 = plt.subplot(5,4,i*4+1), plt.subplot(5,4,i*4+2)
    ax3, ax4 = plt.subplot(5,4,i*4+3), plt.subplot(5,4,i*4+4)
    r = evaluate_planning(TrajEnv(new_dist), ag_new, pos, axes=(ax1, ax2))
    plt.title('Rew: %.2f' % r)
    r = evaluate_planning(TrajEnv(new_dist), ag, pos, axes=(ax3, ax4))
    plt.title('Rew: %.2f' % r)
    if i==4: break
plt.tight_layout()

In [ ]:
angles = np.linspace(0, 2 * np.pi, num=12)
def direction_performance(angles, agent, mag=1, nominal_disturbance=nominal_disturbance):
    radapt, rnom = [], []
    for angle in tqdm(angles, leave=False):
        dx, dy = mag * np.cos(angle), mag * np.sin(angle)
        d = np.asarray([dx, dy], np.float32)
        def new_d(t, pos):
            return d[:ndim]
        anew, info = adapt_traj(nominal_disturbance, new_d, agent)
        rew_nom = evaluate_planning(TrajEnv(new_d), agent, pos, plot=False)
        rew_adapt = evaluate_planning(TrajEnv(new_d), anew, pos, plot=False)
        rnom.append(rew_nom)
        radapt.append(rew_adapt)
    return rnom, radapt

In [ ]:
plt.figure(figsize=(5,12))
plt.subplot(7,1,1)
plt.quiver(np.arange(len(angles)), np.zeros(len(angles)), np.cos(angles), np.sin(angles))

plt.subplot(7,1,(2,3))
rnom, radapt = direction_performance(angles, agent_nominal, 0.5, nominal_disturbance)
plt.plot(rnom, label='Nominal')
plt.plot(radapt, label='Adapted', ls=':')

plt.subplot(7,1,(4,5))
rnom, radapt = direction_performance(angles, agent_nominal, 1.0, nominal_disturbance)
plt.plot(rnom, label='Nominal')
plt.plot(radapt, label='Adapted', ls=':')
plt.legend()

plt.subplot(7,1,(6,7))
rnom, radapt = direction_performance(angles, agent_nominal, 1.5, nominal_disturbance)
plt.plot(rnom, label='Nominal')
plt.plot(radapt, label='Adapted', ls=':')
plt.legend()

### Simulation

In [ ]:
def wind(t, m, nominal=False):
    if nominal:
        return np.zeros(3)
    w_inertial = np.asarray([5 * np.sin(t * 2 * np.pi / 4000), 0, 0])
    dcm = direction_cosine_matrix(*m.orientation)
    return inertial_to_body(w_inertial, dcm)

In [ ]:
def run_sim(env, traj, steps=60_000, disturbance=None):
    ctrl = get_controller(env.vehicle, max_velocity=5.)

    log = DataLog(env.vehicle, ctrl,
                  other_vars=('speeds','target', 'alloc_errs', 'att_err',
                              'rate_target', 'att_target',
                              'leash', 'currents', 'voltages'))
    disturb_force, disturb_torque = 0., 0
    for i, (pos, feed_forward_vel) in tqdm(
        enumerate(traj), leave=False, total=steps
    ):
        if i==steps: break
        # Generate reference for controller
        ref = np.asarray([*pos, 0.])
        # Get prescribed dynamics for system as thrust and torques
        dynamics = ctrl.step(ref, feed_forward_velocity=feed_forward_vel)
        thrust, torques = dynamics[0], dynamics[1:]
        # Allocate control: Convert dynamics into motor rad/s
        action = env.vehicle.allocate_control(thrust, torques)
        # get any disturbances
        if disturbance is not None:
            disturb_force, disturb_torque = disturbance(i, env.vehicle)
        # Send speeds to environment
        state, *_ = env.step(
            action, disturb_forces=disturb_force, disturb_torques=disturb_torque
        )
        alloc_errs = np.asarray([thrust, *torques]) - env.vehicle.alloc @ action**2

        log.log(speeds=action, target=pos, alloc_errs=alloc_errs,
                leash=ctrl.ctrl_p.leash,
                att_err=ctrl.ctrl_a.err,
                att_target = ctrl.ctrl_v.action[::-1],
                rate_target=ctrl.ctrl_a.action,
                currents=[p.motor.current for p in env.vehicle.propellers],
                voltages=[p.motor.voltage for p in env.vehicle.propellers])

        if np.any(np.abs(env.vehicle.orientation[:2]) > np.pi/6): break

    log.done_logging()
    return log

In [ ]:
env = LocalOctorotor(vehicle=Multirotor(vp, sp))

waypoints = [[0,50,2], [50,50,2], [50,0,2], [25,-25,2]]
traj = GuidedTrajectory(env.vehicle, waypoints, proximity=2)
# traj = Trajectory(env.vehicle, waypoints, proximity=2, resolution=None)
log = run_sim(env, traj)

In [ ]:
plt.plot(log.currents, ls=':')
plt.ylabel('Motor current /A')
plt.xlabel('Time /ms')
plt.title('Individual motor currents')

In [ ]:
plt.plot(log.voltages, ls=':')
plt.ylim(0, 30)
plt.ylabel('Motor voltage /A')
plt.xlabel('Time /ms')
plt.title('Voltages')

In [ ]:
plot_trajectory(traj.trajs[-1], dt=0.1)

In [ ]:
%matplotlib inline
plt.figure(figsize=(21,10.5))
plot_grid = (3,3)
plt.subplot(*plot_grid,1)

n = len(log)

plt.plot(log.x, label='x', c='r')
plt.plot(log.target[:, 0], c='r', ls=':')
plt.plot(log.y, label='y', c='g')
plt.plot(log.target[:, 1], c='g', ls=':')
plt.plot(log.z, label='z', c='b')
lines = plt.gca().lines[::2]
plt.ylabel('Position /m')
plt.twinx()
plt.plot(log.roll * (180 / np.pi), label='roll', c='c', ls=':')
plt.plot(log.pitch * (180 / np.pi), label='pitch', c='m', ls=':')
plt.plot(log.yaw * (180 / np.pi), label='yaw', c='y', ls=':')
plt.ylabel('Orientation /deg')
plt.legend(handles=plt.gca().lines + lines, ncol=2)
plt.title('Position and Orientation')

plt.subplot(*plot_grid,2)
for i in range(log.speeds.shape[1]):
    l, = plt.plot(log.speeds[:,i], label='prop %d' % i)
#     plt.plot(speeds[:,i], c=l.get_c())
lines = plt.gca().lines
plt.legend(handles=lines, ncol=2)
plt.title('Motor speeds /RPM')


plt.subplot(*plot_grid,3)
v_world = np.zeros_like(log.velocity)
for i, (v, o) in enumerate(zip(log.velocity, log.orientation)):
    dcm = direction_cosine_matrix(*o)
    v_world[i] = body_to_inertial(v, dcm)
for i, c, a in zip(range(3), 'rgb', 'xyz'):
    plt.plot(v_world[:,i], label='Velocity %s' % a, c=c)
#     plt.plot(velocities[:,i], label='Velocity %s' % a, c=c)
plt.legend()
plt.title('Velocities')

plt.subplot(*plot_grid,4)
plt.title('Controller allocated dynamics')
l = plt.plot(log.actions[:,0], label='Ctrl Thrust')
plt.ylabel('Force /N')
plt.twinx()
for i, c, a in zip(range(3), 'rgb', 'xyz'):
    plt.plot(log.actions[:,1+i], label='Ctrl Torque %s' % a, c=c)
plt.ylabel('Torque /Nm')
plt.legend(handles=plt.gca().lines + l, ncol=2)

plt.subplot(*plot_grid,5)
lines = plt.plot(log.alloc_errs[:, 0], label='Thrust err', c='b')
plt.ylabel('Thrust /N')
plt.twinx()
plt.plot(log.alloc_errs[:, 1], label='Torque x err', ls=':')
plt.plot(log.alloc_errs[:, 2], label='Torque y err', ls=':')
plt.plot(log.alloc_errs[:, 3], label='Torque z err', ls=':')
plt.legend(handles = plt.gca().lines + lines, ncol=2)
plt.ylabel('Torque /Nm')
plt.title('Allocation Errors')

plt.subplot(*plot_grid,6)
plt.plot(log.target[:,0], log.target[:,1], label='Prescribed traj')
plt.plot(log.x, log.y, label='Actual traj', ls=':')
plt.gca().set_aspect('equal', 'box')
plt.title('XY positions /m')
plt.legend()

plt.tight_layout()

In [ ]:
%matplotlib notebook
fig = plt.figure()
xlim = ylim = zlim = (np.min(log.position), np.max(log.position))
ax = fig.add_subplot(projection='3d', xlim=xlim, ylim=ylim, zlim=zlim)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.plot(log.x, log.y, log.z)